In [ ]:

import pandas as pd
from qmt_data import QmtData
import os
import datetime
from xtquant import xtdata

In [ ]:
today = datetime.datetime.today().strftime("%Y%m%d")

In [ ]:
qmt_data = QmtData()
ticker_df = qmt_data.get_tickers()
stock_list = ticker_df["ticker"].tolist()



In [ ]:
for ticker in stock_list:
    df = xtdata.get_divid_factors(ticker, start_time="20000101", end_time="20240820")
    if df.empty:
        continue
    df = (
        df
        .reset_index()
        .rename(columns={"index": "trade_time"})
        .drop(columns=["time"])
    )
    df["trade_time"] = pd.to_datetime(df["trade_time"])
    df.to_parquet(f"D:/qmt_datadir/adjust_factor/{ticker}.parquet", compression="snappy")

In [ ]:
trade_time = pd.DataFrame(xtdata.get_trading_dates("SH"), columns=["trade_time"])
trade_time["trade_time"] = trade_time["trade_time"].apply(lambda s:datetime.fromtimestamp(s / 1000))

100%|██████████| 7770/7770 [00:00<00:00, 178455.61it/s]


D:/qmt_datadir/1m\000001.SH
D:/qmt_datadir/1m\000001.SZ
D:/qmt_datadir/1m\000002.SH
D:/qmt_datadir/1m\000002.SZ
D:/qmt_datadir/1m\000003.SH
D:/qmt_datadir/1m\000004.SH
D:/qmt_datadir/1m\000004.SZ
D:/qmt_datadir/1m\000005.SH
D:/qmt_datadir/1m\000006.SH
D:/qmt_datadir/1m\000006.SZ
D:/qmt_datadir/1m\000007.SH
D:/qmt_datadir/1m\000007.SZ
D:/qmt_datadir/1m\000008.SH
D:/qmt_datadir/1m\000008.SZ
D:/qmt_datadir/1m\000009.SH
D:/qmt_datadir/1m\000009.SZ
D:/qmt_datadir/1m\000010.SH
D:/qmt_datadir/1m\000010.SZ
D:/qmt_datadir/1m\000011.SH
D:/qmt_datadir/1m\000011.SZ
D:/qmt_datadir/1m\000012.SH
D:/qmt_datadir/1m\000012.SZ
D:/qmt_datadir/1m\000013.SH
D:/qmt_datadir/1m\000014.SZ
D:/qmt_datadir/1m\000015.SH
D:/qmt_datadir/1m\000016.SH
D:/qmt_datadir/1m\000016.SZ
D:/qmt_datadir/1m\000017.SH
D:/qmt_datadir/1m\000017.SZ
D:/qmt_datadir/1m\000018.SH
D:/qmt_datadir/1m\000019.SH
D:/qmt_datadir/1m\000019.SZ
D:/qmt_datadir/1m\000020.SH
D:/qmt_datadir/1m\000020.SZ
D:/qmt_datadir/1m\000021.SH
D:/qmt_datadir/1m\00

 67%|██████▋   | 2/3 [00:00<00:00, 13.87it/s]

D:/qmt_datadir/5m\000001.SH
D:/qmt_datadir/5m\000001.SZ
D:/qmt_datadir/5m\000002.SH
D:/qmt_datadir/5m\000002.SZ
D:/qmt_datadir/5m\000003.SH
D:/qmt_datadir/5m\000004.SH
D:/qmt_datadir/5m\000004.SZ
D:/qmt_datadir/5m\000005.SH
D:/qmt_datadir/5m\000006.SH
D:/qmt_datadir/5m\000006.SZ
D:/qmt_datadir/5m\000007.SH
D:/qmt_datadir/5m\000007.SZ
D:/qmt_datadir/5m\000008.SH
D:/qmt_datadir/5m\000008.SZ
D:/qmt_datadir/5m\000009.SH
D:/qmt_datadir/5m\000009.SZ
D:/qmt_datadir/5m\000010.SH
D:/qmt_datadir/5m\000010.SZ
D:/qmt_datadir/5m\000011.SH
D:/qmt_datadir/5m\000011.SZ
D:/qmt_datadir/5m\000012.SH
D:/qmt_datadir/5m\000012.SZ
D:/qmt_datadir/5m\000013.SH
D:/qmt_datadir/5m\000014.SZ
D:/qmt_datadir/5m\000015.SH
D:/qmt_datadir/5m\000016.SH
D:/qmt_datadir/5m\000016.SZ
D:/qmt_datadir/5m\000017.SH
D:/qmt_datadir/5m\000017.SZ
D:/qmt_datadir/5m\000018.SH
D:/qmt_datadir/5m\000019.SH
D:/qmt_datadir/5m\000019.SZ
D:/qmt_datadir/5m\000020.SH
D:/qmt_datadir/5m\000020.SZ
D:/qmt_datadir/5m\000021.SH
D:/qmt_datadir/5m\00

100%|██████████| 3/3 [00:00<00:00, 12.88it/s]

D:/qmt_datadir/1d\000001.SH
D:/qmt_datadir/1d\000001.SZ
D:/qmt_datadir/1d\000002.SH
D:/qmt_datadir/1d\000002.SZ
D:/qmt_datadir/1d\000003.SH
D:/qmt_datadir/1d\000004.SH
D:/qmt_datadir/1d\000004.SZ
D:/qmt_datadir/1d\000005.SH
D:/qmt_datadir/1d\000006.SH
D:/qmt_datadir/1d\000006.SZ
D:/qmt_datadir/1d\000007.SH
D:/qmt_datadir/1d\000007.SZ
D:/qmt_datadir/1d\000008.SH
D:/qmt_datadir/1d\000008.SZ
D:/qmt_datadir/1d\000009.SH
D:/qmt_datadir/1d\000009.SZ
D:/qmt_datadir/1d\000010.SH
D:/qmt_datadir/1d\000010.SZ
D:/qmt_datadir/1d\000011.SH
D:/qmt_datadir/1d\000011.SZ
D:/qmt_datadir/1d\000012.SH
D:/qmt_datadir/1d\000012.SZ
D:/qmt_datadir/1d\000013.SH
D:/qmt_datadir/1d\000014.SZ
D:/qmt_datadir/1d\000015.SH
D:/qmt_datadir/1d\000016.SH
D:/qmt_datadir/1d\000016.SZ
D:/qmt_datadir/1d\000017.SH
D:/qmt_datadir/1d\000017.SZ
D:/qmt_datadir/1d\000018.SH
D:/qmt_datadir/1d\000019.SH
D:/qmt_datadir/1d\000019.SZ
D:/qmt_datadir/1d\000020.SH
D:/qmt_datadir/1d\000020.SZ
D:/qmt_datadir/1d\000021.SH
D:/qmt_datadir/1d\00